# A small test demo of the neuralg eig function

In [ ]:
import neuralg 
import numpy as np
import torch
import networkx # used to generate graphs
from dotmap import DotMap

The module allows for setting precision and potentially employed on GPU 

In [ ]:
neuralg.set_precision("float64")

## Spectral graph theory application

In spectral graph theory, the graph properties are studied in light of the eigendecompositions of their associated matrices. 

In the cell below, we generate $N$ random undirected graphs each with $n$ number of nodes, drawing a possble edge with probability $p$, and save the corresponding Adjacency and Laplacian matrices incl. their eigenvalue spectras. 

In [ ]:
N = 1000
number_of_nodes = np.arange(5,11)
undirected_graphs = DotMap()
for n in number_of_nodes:
    for i in range(N):
        g = networkx.erdos_renyi_graph(n=n, p= 0.5)
        adjacency_matrix = torch.tensor(networkx.to_numpy_array(g))[None,:]
        laplacian_matrix = torch.tensor(networkx.laplacian_matrix(g).toarray(), dtype = torch.float64 )[None,:]
        if i == 0: 
            A = adjacency_matrix
            L = laplacian_matrix
            
        else:   
            A = torch.cat((A,adjacency_matrix),0)
            L = torch.cat((L,laplacian_matrix),0)
    undirected_graphs[n].adjacency = A
    undirected_graphs[n].adjacency.eigvals = neuralg.eig(A)
    undirected_graphs[n].laplacian = L
    undirected_graphs[n].laplacian.eigvals = neuralg.eig(L)
 

#### After using the module, we can clear the loaded models to free the allocated memory

In [ ]:
neuralg.clear_loaded_models()
assert neuralg.neuralg_ModelHandler.loaded_models == {}

## A machine learning task using the neuralg module

Lets say we want a neural network that approximates the total number of length $k$ cycles starting from any node. In exact arithmetic, this quantity is given by 

$$trace(A^k),$$

where $A$ is the adjacency matrix of the graph, but we do not want to perform the $k$ matrix multiplications. From the properties of the trace operator, we can relate this quantity to  the spectrum of $A$ such that  

$$ \# \text{cycles of length } k = \sum_{i} \lambda_i(A)^k $$ 

To this end, we can use the eig function from the neuralg module to approximate the ground truths in the supervised learning of predicting this quantity.  

#### Define a simple convolutional net for the regression task. A convolutional block is followed by two dense layers.

In [ ]:
import torch.nn as nn 
import torch.nn.functional as F

class CycleCNN(nn.Module):
    def __init__(self, n_graph_nodes, conv_layers, filters, kernel_size):
        super(CycleCNN, self).__init__()
        self.net = []
        self.n_graph_nodes = n_graph_nodes
        self.net.append(nn.Conv2d(1,filters,kernel_size, padding = "same"))
        self.net.append(nn.BatchNorm2d(filters))
        self.net.append(nn.ReLU())
        for i in range(conv_layers-1):
            self.net.append(nn.Conv2d(filters,filters,kernel_size, padding = "same"))
            self.net.append(nn.BatchNorm2d(filters))
            self.net.append(nn.ReLU())
        
        self.net.append(nn.Conv2d(filters,1,kernel_size, padding = "same"))
        self.net.append(nn.Flatten())
        self.net.append(DenseLayer(n_graph_nodes**2,n_graph_nodes))
        self.net.append(DenseLayer(n_graph_nodes,1, is_last = True))
        self.net = nn.Sequential(*self.net)
    
    def forward(self, x):
        out = self.net(x)
        return out

class DenseLayer(nn.Module):

    def __init__(self, in_features, out_features, bias=True, is_last = False):
        super().__init__()
        self.is_last = is_last
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)

    def forward(self, input):
        if self.is_last: 
            return self.linear(input)
        else:
            return F.relu(self.linear(input))

#### Define graph generation properties and target cycle length and instantiate model

In [ ]:

#Data parameters 
n_graph_nodes = 5 #This
k = 5 #Cycle length
p = 0.5

#Instantiate model
model = CycleCNN(n_graph_nodes,conv_layers = 3,filters = 32,kernel_size=3)


In [ ]:

import copy
import time

#Training parameters
iterations = 10000
batch_size = 32

criterion = nn.MSELoss()

lr = 3e-4 # learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

model.train(); # turn on train mode


In [ ]:
def get_adjacency_batch(batch_size,n_graph_nodes):
    """Stupid generator of random graph adjacency matrices

    Args:
        batch_size (int): Sife of batch to be generated

    Returns:
        tensor: Tensor of shape [batchsize,1,n_graph_nodes,n_graph_nodes] 
    """
    for i in range(batch_size):
        g = networkx.erdos_renyi_graph(n=n_graph_nodes, p= 0.5)
        adj_matrix = torch.tensor(networkx.to_numpy_array(g))[None,:]
        if i == 0: 
            A = adj_matrix
        else:   
            A = torch.cat((A,adj_matrix),0)
        
    return A[:,None,:]

In [ ]:

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 1000
    start_time = time.time()

    for i in range(iterations):
        
        #Sample a batch
        A = get_adjacency_batch(batch_size, n_graph_nodes= n_graph_nodes)
    

        #Predict # of k-cycles with model
        output = model(A)

        #Use module to compute ground truth
        target_eigvals = neuralg.eig(A)
        target = torch.pow(target_eigvals,k).sum(-1)
        
        loss = criterion(output,target)
    
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()

        total_loss += loss.item()
        
        if i % log_interval == 0 and i > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval

            print(f'| epoch {epoch:3d} | {i:5d} batches | '
                  f'lr {lr:02.4f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.5f}')

            total_loss = 0
            start_time = time.time()

In [ ]:
epochs = 10
best_model = None
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)

    scheduler.step()

#### Some preliminary evaluation from training
We can use the previously generated graphs to evaluate the trained model. We also include the eigenvalue ground truths using the torch module 

In [ ]:
eval_set = undirected_graphs[n_graph_nodes].adjacency[:,None,:] #A dummy index for the channel dimension is added

eval_pred = torch.round(model(eval_set)) #Since path length should be an integer, we round the predictions
neuralg_targets = torch.round(torch.pow(neuralg.eig(eval_set),k).sum(-1)) #Compute ground truth with neuralg module
torch_targets = torch.round(torch.pow(torch.real(torch.linalg.eigvals(eval_set)),k).sum(-1)) #Compute ground truth with torch built-int 

In [ ]:
from neuralg.evaluation.compute_accuracy import compute_accuracy
from neuralg.training.losses import relative_L1_evaluation_error

neuralg_eval_errors= relative_L1_evaluation_error(eval_pred,neuralg_targets)
torch_eval_errors = relative_L1_evaluation_error(eval_pred,torch_targets) 

tol = 0.1

neuralg_acc = compute_accuracy(tol,neuralg_eval_errors).item()
torch_acc = compute_accuracy(tol,torch_eval_errors).item()

print("Tolerance set to {:.0%}".format(tol))
print("Test accuracy with neuralg ground truths = {}".format(neuralg_acc))
print("Test accuracy with torch ground truths = {}" .format(torch_acc))